In [1]:
#%load_ext autoreload
#%autoreload 2

#import sys
#sys.path.insert(0,'..')

import os

import backtrader as bt
import backtrader.indicators as btind
import numpy as np
import scipy.signal as signal
from scipy import stats

from gym import spaces

from btgym import BTgymEnv, BTgymStrategy, BTgymDataset

from launcher import Launcher
from model import LSTMPolicy

In [4]:
# GYM TEST ENV:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing_gym',
)

env_config = dict(
    gym_id='Breakout-v0'
)

class LSTM_new(LSTMPolicy):
    pass

launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    model_class=LSTM_new,
    train_steps=500000000,
    opt_learn_rate=1e-4,
    rollout_length=20,
    test_mode=True,
    model_summary_freq=50,
    episode_summary_freq=2,
    env_render_freq=10,
    verbose=2
)

[2017-08-24 13:50:37,143] Launcher ready.


In [5]:
launcher.run()


[2017-08-24 13:50:38,568] parameters_server started.
[2017-08-24 13:50:38,572] worker_0 tf.server started.
[2017-08-24 13:50:38,575] making environment.
[2017-08-24 13:50:38,577] Making new env: Breakout-v0
[2017-08-24 13:50:38,770] worker_0:envronment ok.
[2017-08-24 13:50:40,699] worker_0:trainer ok.
[2017-08-24 13:50:41,828] connecting to the parameter server... 


INFO:tensorflow:Restoring parameters from ./tmp/a3c_testing_gym/train/model.ckpt-0


[2017-08-24 13:50:41,836] Restoring parameters from ./tmp/a3c_testing_gym/train/model.ckpt-0
[2017-08-24 13:50:43,581] worker_1 tf.server started.
[2017-08-24 13:50:43,586] worker_2 tf.server started.
[2017-08-24 13:50:43,586] making environment.
[2017-08-24 13:50:43,592] making environment.
[2017-08-24 13:50:43,593] worker_3 tf.server started.
[2017-08-24 13:50:43,601] Press `Ctrl-C` to stop training and close launcher.
[2017-08-24 13:50:43,599] Making new env: Breakout-v0
[2017-08-24 13:50:43,604] Making new env: Breakout-v0
[2017-08-24 13:50:43,604] making environment.
[2017-08-24 13:50:43,599] worker_4 tf.server started.
[2017-08-24 13:50:43,629] making environment.
[2017-08-24 13:50:43,608] worker_5 tf.server started.
[2017-08-24 13:50:43,622] Making new env: Breakout-v0
[2017-08-24 13:50:43,620] worker_6 tf.server started.
[2017-08-24 13:50:43,626] worker_7 tf.server started.
[2017-08-24 13:50:43,631] making environment.
[2017-08-24 13:50:43,640] making environment.
[2017-08-24 1

Press `Ctrl-C` to stop training and close launcher.


[2017-08-24 13:50:47,559] worker_3:trainer ok.
[2017-08-24 13:50:47,574] worker_7:trainer ok.
[2017-08-24 13:50:47,610] worker_2:trainer ok.
[2017-08-24 13:50:47,607] worker_6:trainer ok.
[2017-08-24 13:50:47,659] worker_4:trainer ok.
[2017-08-24 13:50:47,664] worker_5:trainer ok.
[2017-08-24 13:50:47,691] worker_1:trainer ok.


...

[2017-08-24 13:50:52,104] worker_7: starting training at step: 459
[2017-08-24 13:50:52,089] Starting queue runners.
[2017-08-24 13:50:52,134] worker_4: starting training at step: 459
[2017-08-24 13:50:52,168] worker_6: starting training at step: 459


INFO:tensorflow:Starting queue runners.


[2017-08-24 13:50:52,381] Starting queue runners.
[2017-08-24 13:50:52,470] worker_1: starting training at step: 499


INFO:tensorflow:global/global_step/sec: 514.502


[2017-08-24 13:52:47,842] global/global_step/sec: 514.502
[2017-08-24 13:53:50,148] worker_1 has joined.
[2017-08-24 13:53:50,155] worker_2 has joined.
[2017-08-24 13:53:50,163] worker_3 has joined.
[2017-08-24 13:53:50,164] worker_4 has joined.
[2017-08-24 13:53:50,165] worker_5 has joined.
[2017-08-24 13:53:50,167] worker_6 has joined.
[2017-08-24 13:53:50,167] worker_7 has joined.
[2017-08-24 13:53:50,168] chief_worker_0 has joined.
[2017-08-24 13:53:50,169] parameter_server_0 has joined.
[2017-08-24 13:53:50,170] Launcher closed.


In [8]:
class MyStrategy(BTgymStrategy):
    """
    Example subclass of BT server inner computation startegy.
    """
    
    def __init__(self, **kwargs):
        super(MyStrategy,self).__init__(**kwargs)
        
        self.order_penalty = 0.5
        self.trade_just_closed = False
        self.trade_result = None
        
    def notify_trade(self, trade):
        #if trade.justopened:
            #print('trade {} just opened'.format(trade.ref))
            
        if trade.isclosed:
            #print('trade {} closed, pnl_comm: {}'.format(trade.ref, trade.pnlcomm))
            # Set trade flag and result:
            self.trade_just_closed = True
            self.trade_result = trade.pnlcomm
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
        
    def get_state(self):
        """
        Computes featurized RL-ready environment observation state
        by applying continious wavelet transform to time-embedded vector
        of close-price gradients.
        """
        # Use close price:
        channel = 3
        
        X = self.raw_state[:, channel]
        
        # Prepare parameters:
        Tau = 5
        max_cwt_scale = self.p.state_shape['model_input'].shape[1]
        cwt_width = np.linspace(Tau, max_cwt_scale + Tau - 1, max_cwt_scale) # scale of wavelet transdorm [n]
    
        T = 1# 000
        
        # Get vector of gradients of last [n] prices:
        X = np.gradient(X, axis=0) * T
        
        # Compute continious wavelet transform using Ricker wavelet, get [n,m,1]-dim. matrix:
        X = signal.cwt(X, signal.ricker, cwt_width).T[:, :, None]
        
        # Z-score:
        X = stats.zscore(X)
        
        # Local min-max norm:
        self.state['model_input'] = (X - X.min()) / (X.max() - X.min())
        
        # Squash values in [0,1]:
        #self.state['model_input'] = self.sigmoid(self.state['model_input'])
        
        return self.state

    
    def get_reward(self):
        """
        Defines reward as [0,1]-bounded function of last closed trade result.
        """
        r = 0
        
        # Result
        if self.trade_just_closed:
            r = self.trade_result
            self.trade_just_closed = False
            
        # Penalty for failed order:
        if self.order_failed:
            #print('Failed order!')
            r -= self.order_penalty
            self.order_failed = False
            
        #print('reward_', r)
        
        return r / 20
            
# Set backtesting engine parameters:

state_shape = {
    'raw_state': spaces.Box(low=-1, high=1, shape=(120, 4)),
    'model_input': spaces.Box(low=-10, high=10, shape=(120, 15, 1))
}

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    MyStrategy,
    state_shape=state_shape,
    portfolio_actions=('hold', 'buy', 'sell'),
    drawdown_call=5, # in percent of initial cash
    target_call=50,
    skip_frame=10,
)

# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0)
MyCerebro.broker.set_shortcash(False)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10000,)


MyCerebro.addanalyzer(bt.analyzers.DrawDown)

# Provide data (seven years of 1 minute bars):
filenames = [
    '../data/DAT_ASCII_EURUSD_M1_2010.csv',
    '../data/DAT_ASCII_EURUSD_M1_2011.csv',
    '../data/DAT_ASCII_EURUSD_M1_2012.csv',
    '../data/DAT_ASCII_EURUSD_M1_2013.csv',
    '../data/DAT_ASCII_EURUSD_M1_2014.csv',
    '../data/DAT_ASCII_EURUSD_M1_2015.csv',
    '../data/DAT_ASCII_EURUSD_M1_2016.csv',
]

MyDataset = BTgymDataset(
    filename=filenames,
    #filename='../data/test_sine_1min_period256_delta0002.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=55,
    start_00=True,
    time_gap_hours=6,
)
env_config = dict(
    dataset=MyDataset,
    engine=MyCerebro,
    render_modes=['episode', 'human', 'model_input'],
    render_state_as_image=True,
    render_ylabel='Z-norm/CWT',
    render_size_episode=(12,8),
    render_size_human=(8, 3.5),
    render_size_state=(10, 5),
    render_dpi=75,
    port=5000,
    data_port=4999,
    connect_timeout=60,
    verbose=0,
)
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=8,
    num_ps=1,
    log_dir='./tmp/a3c_testing',
)
launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    rollout_length=50,
    test_mode=False,
    train_steps=1000000000,
    model_summary_freq=20,
    episode_summary_freq=1,
    env_render_freq=20,
    verbose=1
    
)

[2017-08-24 14:02:30,570] port 12223 cleared


In [ ]:
launcher.run()

[2017-08-24 14:02:37,165] Press `Ctrl-C` to stop training and close launcher.


Press `Ctrl-C` to stop training and close launcher.
INFO:tensorflow:Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-92425


[2017-08-24 14:02:45,004] Restoring parameters from ./tmp/a3c_testing/train/model.ckpt-92425


INFO:tensorflow:Waiting for model to be ready.  Ready_for_local_init_op:  None, ready: Variables not initialized: global/l1/W, global/l1/b, global/l2/W, global/l2/b, global/l3/W, global/l3/b, global/l4/W, global/l4/b, global/rnn/basic_lstm_cell/kernel, global/rnn/basic_lstm_cell/bias, global/action/w, global/action/b, global/value/w, global/value/b, global/global_step, global/global_episode, beta1_power, beta2_power, global/l1/W/Adam, global/l1/W/Adam_1, global/l1/b/Adam, global/l1/b/Adam_1, global/l2/W/Adam, global/l2/W/Adam_1, global/l2/b/Adam, global/l2/b/Adam_1, global/l3/W/Adam, global/l3/W/Adam_1, global/l3/b/Adam, global/l3/b/Adam_1, global/l4/W/Adam, global/l4/W/Adam_1, global/l4/b/Adam, global/l4/b/Adam_1, global/rnn/basic_lstm_cell/kernel/Adam, global/rnn/basic_lstm_cell/kernel/Adam_1, global/rnn/basic_lstm_cell/bias/Adam, global/rnn/basic_lstm_cell/bias/Adam_1, global/action/w/Adam, global/action/w/Adam_1, global/action/b/Adam, global/action/b/Adam_1, global/value/w/Adam, gl

[2017-08-24 14:02:46,277] Waiting for model to be ready.  Ready_for_local_init_op:  None, ready: Variables not initialized: global/l1/W, global/l1/b, global/l2/W, global/l2/b, global/l3/W, global/l3/b, global/l4/W, global/l4/b, global/rnn/basic_lstm_cell/kernel, global/rnn/basic_lstm_cell/bias, global/action/w, global/action/b, global/value/w, global/value/b, global/global_step, global/global_episode, beta1_power, beta2_power, global/l1/W/Adam, global/l1/W/Adam_1, global/l1/b/Adam, global/l1/b/Adam_1, global/l2/W/Adam, global/l2/W/Adam_1, global/l2/b/Adam, global/l2/b/Adam_1, global/l3/W/Adam, global/l3/W/Adam_1, global/l3/b/Adam, global/l3/b/Adam_1, global/l4/W/Adam, global/l4/W/Adam_1, global/l4/b/Adam, global/l4/b/Adam_1, global/rnn/basic_lstm_cell/kernel/Adam, global/rnn/basic_lstm_cell/kernel/Adam_1, global/rnn/basic_lstm_cell/bias/Adam, global/rnn/basic_lstm_cell/bias/Adam_1, global/action/w/Adam, global/action/w/Adam_1, global/action/b/Adam, global/action/b/Adam_1, global/value/

...

[2017-08-24 14:03:18,117] Starting queue runners.


INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Starting queue runners.


[2017-08-24 14:03:18,105] Starting queue runners.
[2017-08-24 14:03:18,123] Starting queue runners.
[2017-08-24 14:03:18,111] Starting queue runners.
[2017-08-24 14:03:18,123] Starting queue runners.
[2017-08-24 14:03:18,174] worker_1: starting training at step: 93536
[2017-08-24 14:03:18,194] worker_6: starting training at step: 93536
[2017-08-24 14:03:18,205] worker_4: starting training at step: 93536
[2017-08-24 14:03:18,205] worker_2: starting training at step: 93536
[2017-08-24 14:03:18,210] worker_5: starting training at step: 93536


INFO:tensorflow:Starting queue runners.


[2017-08-24 14:03:18,206] Starting queue runners.
[2017-08-24 14:03:18,268] worker_7: starting training at step: 93536


In [ ]:
print(launcher.kwargs, '\n\n')
print(launcher.env_config)
print(launcher.cluster_config)
print(launcher.cluster_spec)
for config in launcher.workers_config_list:
    print('============')
    for k, v in config.items():
        print('{}:\n{}\n'.format(k, v))
    

In [ ]:
def func1(max_step):
    step = 0
    done = False
    
    def func2(max_step):
        nonlocal step
        nonlocal done
        step +=1
        if step == max_step:
            step = 0
            done = True
        return step
    
    for i in range(20):
        done = False
        print(func2(max_step), step, done)
        


func1(7)
            

In [ ]:
a = dict()
a.update({'b': 2, 'c':4})
type(a) == dict